### This nodebook is simply a sanity check to verify zero rates calibration by pricing a series of ZCB using the tree and checking the difference between the actual/theoretical price given by the ZCB curve.

In [ ]:
import numpy as np
from HullWhite import OneFactorHullWhiteModel
from HullWhiteTrinomialTree import OneFactorHullWhiteTrinomialTree, Node
from ZeroRateCurve import LinearZeroRateCurve, StepwiseZeroRateCurve
from HullWhiteTreeZCBPricer import HullWhiteTreeZCBPricer

payment_times = [0, 40]
model = OneFactorHullWhiteModel(0.03, 0.005)
# zcb_curve = LinearZeroRateCurve(0.0, 0.05, 30)
zcb_curve = StepwiseZeroRateCurve()

# set up and build tree
tree = OneFactorHullWhiteTrinomialTree(model, payment_times, zcb_curve, 0.25)
tree.build_tree(verbose=False)

# set up pricer
zcb_pricer = HullWhiteTreeZCBPricer(tree=tree)

# loop
cur_layer = tree.root_node.layer_attr
closed_form_price = []
tree_price = []
while cur_layer:
    m = cur_layer.layer_id
    P = np.exp( -1 * cur_layer.t * zcb_curve.get_zero_rate(cur_layer.t) )
    P_tree = zcb_pricer.price(cur_layer.t, 1)
    closed_form_price.append(P)
    tree_price.append(P_tree)

    cur_layer = cur_layer.next_layer_attr

diff = np.array(tree_price) - np.array(closed_form_price)
diff = diff ** 2
diff = list([round(float(se), 8) for se in diff])
diff

Building tree...
Calibrating the tree to zero curve...
Tree built successfully.


KeyboardInterrupt: 